In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from read_data import get_dirs
import os

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image

In [8]:
class PhotoDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row["path"]).convert("RGB")
        
        if self.transform:
            image = self.transform(image)

        label = torch.tensor(row["label"], dtype=torch.long)
        return image, label


In [ ]:
transform = transforms.Compose([
    # change image size
    transforms.Resize((224, 224)),
    # converts PIL image to PyTorch tensor
    transforms.ToTensor()
])


In [ ]:
df = get_dirs()

# initialising every image transformed
dataset = PhotoDataset(df, transform=transform)

# data wrapper for convenient usage to model
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

# 16 images/labels, 3 colors, 224 height, 224 width
# 16 at a time
images, labels = next(iter(train_loader))
print(images.shape, labels.shape)



torch.Size([16, 3, 224, 224]) torch.Size([16])


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using:", device)

# resnet18 model with default weights (pre-trained IMAGENET1K_V1)
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# X*W + c, using final layer nodes to output ???
model.fc = nn.Linear(model.fc.in_features, 2)   # Alex vs Kelly
model = model.to(device)

Using: cpu
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/sumanth/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100.0%


In [ ]:
criterion = nn.CrossEntropyLoss()

# optimizer on weights using learning rate 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
model.train()

for batch_idx, (imgs, lbls) in enumerate(train_loader):
    imgs, lbls = imgs.to(device), lbls.to(device)

    # clear old gradients
    optimizer.zero_grad()

    # output of model
    outputs = model(imgs)

    # calculate loss based on current output/weights
    loss = criterion(outputs, lbls)

    # backward propagation to train model
    loss.backward()

    # forward propagation to update parameters
    optimizer.step()

    print(f"Batch {batch_idx}: Loss = {loss.item():.4f}")

    if batch_idx == 5:
        break


Batch 0: Loss = 0.7322
Batch 1: Loss = 0.6119
Batch 2: Loss = 0.5328
Batch 3: Loss = 0.6830
Batch 4: Loss = 0.4426
Batch 5: Loss = 0.5724
